In [2]:
from torch import nn
import torch
import sys
sys.path.append('..')
from mae.models_mae import MaskedAutoencoderViT, partial
sys.path.append('../..')
from DUL294P.encoders.openclip_encoder import *


%load_ext autoreload
%autoreload 2

/home/sandeepmukh/DeepUL294Proj/.deepulenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
clipencoder = OpenCLIPNetworkConfig(device='cpu').setup()

In [10]:
model = MaskedAutoencoderViT(
        patch_size=16, embed_dim=768, depth=12, num_heads=12,
        decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
        mlp_ratio=4, out_chans=768, norm_layer=partial(nn.LayerNorm, eps=1e-6), return_pooled=True)
sd=torch.load('../models/mae_pretrain_vit_base_full.pth')['model']
msg = model.load_state_dict(sd, strict=False)
print(msg)

img = torch.randn(1, 3, 224, 224)
target = model.patchify(img)
latent, mask, ids_restore = model.forward_encoder(img, .75)
pred, pooled = model.forward_decoder(latent, ids_restore)
pred.shape, pooled.shape

_IncompatibleKeys(missing_keys=['pool_proj.weight', 'pool_proj.bias'], unexpected_keys=[])


(torch.Size([1, 196, 768]), torch.Size([1, 512]))

In [4]:
clipencoder.model.encode_text(clipencoder.tokenizer(['a photo of a cat', 'nofdnjk']), normalize=True).shape

torch.Size([2, 512])

In [12]:
clipencoder.process

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
    Normalize(mean=[0.48145466, 0.4578275, 0.40821073], std=[0.26862954, 0.26130258, 0.27577711])
)

In [14]:
pred.norm(dim=-1)

tensor([[35.9688, 26.9219, 27.5469, 26.9531, 26.4844, 27.8750, 28.6562, 27.5312,
         27.2344, 28.4688, 27.3594, 27.8594, 41.1562, 28.4688, 28.2188, 26.7344,
         29.3594, 28.0469, 39.6250, 40.3438, 31.9531, 28.2031, 26.8750, 27.2031,
         26.1875, 28.1094, 27.6562, 28.1719, 29.8750, 27.7500, 26.7812, 28.7812,
         28.3750, 26.7344, 28.2656, 26.8438, 28.1094, 26.6562, 28.5469, 27.5000,
         27.6875, 27.7344, 28.1094, 28.9219, 26.4531, 27.4531, 27.0625, 43.6562,
         28.2969, 27.8906, 27.5312, 28.8281, 29.6406, 28.6250, 27.5938, 29.4844,
         29.3750, 29.1406, 28.2500, 28.6562, 29.4844, 28.4531, 28.8750, 28.0000,
         27.4844, 28.0625, 27.1719, 27.9219, 28.0469, 28.6562, 28.0156, 29.8750,
         27.0156, 29.0000, 28.9531, 27.4375, 28.0000, 28.1562, 28.6562, 27.8281,
         28.0625, 28.1094, 28.8281, 26.1094, 28.3281, 26.4531, 28.5625, 26.9844,
         29.8281, 28.6406, 27.3750, 27.3594, 27.9062, 27.8125, 26.9219, 29.8750,
         27.1094, 27.7188, 2

: 

In [13]:
img_prepro = clipencoder.process(img).half()
pooled, pred = clipencoder.model.encode_image(img_prepro)
pred.shape, pooled.shape

(torch.Size([1, 196, 768]), torch.Size([1, 512]))